# Post-process vasca_pipe results
Make catalog file of selected sources only, associate and get store additional info for listed sources

In [3]:
region_name = "ALL_10-800_LOOSE" #"CAINGSGII_10-800"  #"TDS" #"CAINGSGII_10-800"  #"TDS" # "MDIS_10-800" # "TDS" # _ELAISN1
region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+".fits"
region_cat_fname = region_fname.replace(".fits","_cat.fits")
do_cat = False                        # Recreate catalog file from vasca_pipe output region
do_cross_match = False                  # Does source matching with CDS
do_src_prep = True                   # Gets additional info for sources listes in srcs_ids list below.
do_variable_addition = False            # Adds variables of matched sources to tt_sources

srcs_ids = []
srcs_ids = [4993 ,13580 ,14504 ,30475 ,50388 ,52994 ,79477 ,120265 ,138783 ,139480 ,162697 ,204419 ,221349 ,228958 ,236839 ,253214 ,
            253670 ,256433 ,261213 ,331466 ,340464 ,381336 ,390864 ,489430 ,498065 ,525310 ,532554 ,533185 ,540235 ,583838 ,639720 ,
            645215 ,645730 ,659787 ,666224 ,673175 ,680119 ,691408 ,714140 ,764827 ,1037284 ,1069077 ,1070938 ,1805505 ,1839206
            ,2527679 ,3074917 ,] # WD ALL_10-800-LOOSE

In [4]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

from astropy.coordinates import SkyCoord
from astropy.table import unique, Table, hstack, join, setdiff
from astropy import units as uu
from astroquery.simbad import Simbad
from astropy.visualization import quantity_support
from astropy.modeling.models import BlackBody
from astropy import constants as cc
from matplotlib.ticker import ScalarFormatter
from astroquery.sdss import SDSS

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis
from vasca.tables import TableCollection, dd_vasca_columns
from vasca.utils import otype2ogroup,dd_ogrp2otypes, dd_ogrp2col, dd_filter2wavelength, mag2flux, flux2mag, query_vizier_sed, add_ogrp

from loguru import logger
logger.enable("vasca")

# Do catalog file

if do_cat:
    rg = Region()
    rg.load_from_fits(region_fname)
    rc = rg.get_region_catalog()
else:
    rc = Region()
    rc.load_from_fits(region_cat_fname)

#Cross match to SIMBAD and GAIA
if do_cross_match:
    rc.cross_match_cds(query_table="simbad")
    rc.cross_match_cds()

#Add variables
if do_variable_addition:
    rc.add_column("tt_gaiadr3", "Gmag_abs")
    sel_plx = rc.tt_gaiadr3["RPlx"]>3
    rc.tt_gaiadr3["Gmag_abs"][sel_plx]=rc.tt_gaiadr3["Gmag"][sel_plx] + 5.0 * np.log10(rc.tt_gaiadr3["Plx"][sel_plx]) - 10.0

    # Add object groups
    add_ogrp(rc.tt_simbad)
    add_ogrp(rc.tt_gaiadr3,provenance="GAIA")

#Write out all changes
rc.write_to_fits(region_cat_fname, overwrite=True)

#Prepare sources, including Vizier SED.
if do_src_prep:
    for src_id in srcs_ids:
        tc_src = rc.get_src_from_id(src_id, load_from_file = False)
print("Done")

2023-10-18 12:28:27.285 | DEBUG    | vasca.tables:load_from_fits:328 - Loading file with name './vasca_pipeline/ALL_10-800_LOOSE/region_ALL_10-800_LOOSE_cat.fits'
2023-10-18 12:28:27.293 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_fields'
2023-10-18 12:28:27.305 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_filters'
2023-10-18 12:28:27.310 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_visits'
2023-10-18 12:28:27.316 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_sources'
2023-10-18 12:28:27.335 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_detections'
2023-10-18 12:28:27.367 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_coadd_detections'
2023-10-18 12:28:27.443 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_coadd_sources'
2023-10-18 12:28:27.458 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_src_id_map'
2023-10-18 12:28:27.465 | DEBUG   

Nr of spectra found 1


2023-10-18 12:28:59.735 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_visits'
2023-10-18 12:28:59.740 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_fields'
2023-10-18 12:28:59.751 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_filters'
2023-10-18 12:28:59.755 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_source_lc'
2023-10-18 12:28:59.765 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_sed'
2023-10-18 12:28:59.771 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_gphoton_lc'
2023-10-18 12:28:59.779 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_spectrum'
2023-10-18 12:28:59.888 | DEBUG    | vasca.region:get_src_from_id:337 - Getting Source object for rg_src_id: 14504
2023-10-18 12:28:59.889 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_sources'
2023-10-18 12:28:59.913 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_detections'
2023-10-18 12:28:59

Nr of spectra found 1


2023-10-18 12:29:18.407 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_visits'
2023-10-18 12:29:18.412 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_fields'
2023-10-18 12:29:18.423 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_filters'
2023-10-18 12:29:18.427 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_source_lc'
2023-10-18 12:29:18.437 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_sed'
2023-10-18 12:29:18.442 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_spectrum'
2023-10-18 12:29:18.539 | DEBUG    | vasca.region:get_src_from_id:337 - Getting Source object for rg_src_id: 30475
2023-10-18 12:29:18.542 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_sources'
2023-10-18 12:29:18.566 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_detections'
2023-10-18 12:29:18.572 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_simbad'
2023-10-18 12:29:18.584 | DE

...

Nr of spectra found 1


2023-10-18 12:31:45.497 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_visits'
2023-10-18 12:31:45.503 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_fields'
2023-10-18 12:31:45.513 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_filters'
2023-10-18 12:31:45.516 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_source_lc'
2023-10-18 12:31:45.525 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_sed'
2023-10-18 12:31:45.531 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_gphoton_lc'
2023-10-18 12:31:45.539 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_spectrum'
2023-10-18 12:31:45.648 | DEBUG    | vasca.region:get_src_from_id:337 - Getting Source object for rg_src_id: 236839
2023-10-18 12:31:45.650 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_sources'
2023-10-18 12:31:45.674 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_detections'
2023-10-18 12:31:4

Nr of spectra found 2


2023-10-18 12:38:08.078 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_detections'
2023-10-18 12:38:08.089 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_simbad'
2023-10-18 12:38:08.107 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_gaiadr3'
2023-10-18 12:38:08.120 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_visits'
2023-10-18 12:38:08.125 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_fields'
2023-10-18 12:38:08.126 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_filters'
2023-10-18 12:38:08.127 | DEBUG    | vasca.tables:get_light_curve:568 - Getting light curve for fd_src_ids 'None' and rg_src_ids '1839206'
2023-10-18 12:38:08.127 | DEBUG    | vasca.tables:get_light_curve:589 - Getting lightcurve for rg_src_id: [1839206]
2023-10-18 12:38:08.147 | DEBUG    | vasca.tables:table_from_template:121 - Created new table from template 'base_field:tt_source_lc'.
2023-10-18 12:38:08.147 | DEBUG    | vasca.tables:add_tabl

Done
